In [27]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# fix random seed for reproducibility
np.random.seed(7)

In [2]:
files = map(np.load, glob.glob("../export/data/feature_data_*.npz"))

In [3]:
signals = files[0]["signals"]
attributes = files[0]["attributes"]
print signals
print attributes

['ECG' 'EDA' 'Resp' 'SKT']
['mean' 'std' 'min' 'max' 'mean_diff' 'mean_abs_diff']


In [4]:
selected_signals = ['ECG', 'EDA',  'Resp', 'SKT']
selected_attribute = ['mean', 'std', 'min', 'max', 'mean_diff', 'mean_abs_diff']

n_selected_signals = len(selected_signals)
n_selected_attribute = len(selected_attribute)

n_subject = len(files)

skip_sample = 100
number_of_sample = 1500

In [5]:
def get_data():
    
    inputs_raw = pd.DataFrame()
    labels = []
    
    for f in files:

        session = f["session_info"].tolist()
        mediafile = session["mediaFile"]
        sid = int(session["sessionId"])
        label = int(session["feltVlnc"])

        data = pd.DataFrame()

        for signal in selected_signals:
            for attribute in selected_attribute:

                # add a column for each feature
                column = signal + "_" + attribute
                data[column] = f["valence"].tolist()[column][skip_sample:skip_sample + number_of_sample]

        # concatenate all subjects' data
        inputs_raw = pd.concat([inputs_raw, data])
        
        # add a label for each timestep
        labels.append(label)

    # reshape to CNN input shape
    X = inputs_raw
    X = X.fillna(X.median())
    X = X.values.reshape(n_subject, number_of_sample, n_selected_attribute*n_selected_signals)

    # reduce class numbers
    Y = pd.DataFrame(labels)

    Y[Y==1] = 0
    Y[Y==2] = 0
    Y[Y==3] = 0
    Y[Y==5] = 0
    Y[Y==7] = 0
    Y[Y==12] = 0

    Y[Y==4] = 1
    Y[Y==6] = 1
    Y[Y==8] = 1
    Y[Y==9] = 1
    Y[Y==11] = 1

    return X, Y

In [6]:
X, Y = get_data()

In [8]:
# split training and validation data
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, shuffle=False)
print x_train.shape
print y_train.shape
print x_valid.shape
print y_valid.shape

(89, 1500, 24)
(89, 1)
(23, 1500, 24)
(23, 1)


In [28]:
sample_weight_train = class_weight.compute_sample_weight('balanced', y_train)
sample_weight_valid = class_weight.compute_sample_weight('balanced', y_valid)

In [24]:
batch_size = x_train.shape[0]
data_dim = x_train.shape[2]
timesteps = x_train.shape[1]

n_classes = y_train.shape[1]

hidden_size = 24
n_epochs = 10

# Model LSTM

In [11]:
model = Sequential()
model.add(LSTM(hidden_size, return_sequences=True, stateful=True, batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(n_classes))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (89, 1500, 100)           50000     
_________________________________________________________________
lstm_4 (LSTM)                (89, 1)                   408       
_________________________________________________________________
dense_2 (Dense)              (89, 1)                   2         
Total params: 50,410
Trainable params: 50,410
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, shuffle=False)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
89/89 [==============================] - 5s 62ms/step - loss: 11.6433 - acc: 0.2697
Epoch 2/10
89/89 [==============================] - 3s 34ms/step - loss: 11.6433 - acc: 0.2697
Epoch 3/10
89/89 [==============================] - 3s 32ms/step - loss: 11.6433 - acc: 0.2697
Epoch 4/10
89/89 [==============================] - 3s 34ms/step - loss: 11.6433 - acc: 0.2697
Epoch 5/10
89/89 [==============================] - 3s 32ms/step - loss: 11.6433 - acc: 0.2697
Epoch 6/10
89/89 [==============================] - 3s 33ms/step - loss: 11.6433 - acc: 0.2697
Epoch 7/10
89/89 [==============================] - 3s 32ms/step - loss: 11.6433 - acc: 0.2697
Epoch 8/10
89/89 [==============================] - 3s 33ms/step - loss: 11.6433 - acc: 0.2697
Epoch 9/10
89/89 [==============================] - 3s 32ms/step - loss: 11.6433 - acc: 0.2697
Epoch 10/10
89/89 [==============================] - 3s 32ms/step - loss: 11.6433 - acc: 0.2697


In [ ]:
model.evaluate(x_valid, y_valid, batch_size=1)

# Model Time distributed LSTM

In [31]:
model_1 = Sequential()
model_1.add(LSTM(hidden_size, return_sequences=True, stateful=True, batch_input_shape=(batch_size, timesteps, data_dim)))
model_1.add(Dropout(rate=1-0.5))
model_1.add(LSTM(hidden_size, return_sequences=True))
model_1.add(Dropout(rate=1-0.5))
model_1.add(LSTM(hidden_size))
model_1.add(Dropout(rate=1-0.5))
model_1.add(Dense(n_classes, activation='softmax'))
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (89, 1500, 24)            4704      
_________________________________________________________________
dropout_22 (Dropout)         (89, 1500, 24)            0         
_________________________________________________________________
lstm_27 (LSTM)               (89, 1500, 24)            4704      
_________________________________________________________________
dropout_23 (Dropout)         (89, 1500, 24)            0         
_________________________________________________________________
lstm_28 (LSTM)               (89, 24)                  4704      
_________________________________________________________________
dropout_24 (Dropout)         (89, 24)                  0         
_________________________________________________________________
dense_10 (Dense)             (89, 1)                   25        
Total para

In [32]:
model_1.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, shuffle=False, sample_weight=sample_weight_train)

Epoch 1/10
89/89 [==============================] - 6s 67ms/step - loss: 7.9712 - acc: 0.2697
Epoch 2/10
89/89 [==============================] - 2s 23ms/step - loss: 7.9712 - acc: 0.2697
Epoch 3/10
89/89 [==============================] - 2s 23ms/step - loss: 7.9712 - acc: 0.2697
Epoch 4/10
89/89 [==============================] - 2s 23ms/step - loss: 7.9712 - acc: 0.2697
Epoch 5/10
89/89 [==============================] - 2s 22ms/step - loss: 7.9712 - acc: 0.2697
Epoch 6/10
89/89 [==============================] - 2s 23ms/step - loss: 7.9712 - acc: 0.2697
Epoch 7/10
89/89 [==============================] - 2s 22ms/step - loss: 7.9712 - acc: 0.2697
Epoch 8/10
89/89 [==============================] - 2s 23ms/step - loss: 7.9712 - acc: 0.2697
Epoch 9/10
89/89 [==============================] - 2s 23ms/step - loss: 7.9712 - acc: 0.2697
Epoch 10/10
89/89 [==============================] - 2s 22ms/step - loss: 7.9712 - acc: 0.2697


In [ ]:
model_1.evaluate(x_valid, y_valid, batch_size=1)